## Reconstruct new models without embedding

In [2]:
from megnet.models import MEGNetModel
from megnet.data.graph import GaussianDistance
from megnet.data.crystal import CrystalGraph
import numpy as np

# Pretrained model
model_pretrained = MEGNetModel.from_file('../mvl_models/'
                                         'mp-2018.6.1/band_gap_regression.hdf5')

# New model with length 16 atom feature
cg = CrystalGraph(bond_converter=GaussianDistance(np.linspace(0, 5, 100), 0.5), cutoff=4)
model_new = MEGNetModel(100, 2, 16, graph_converter=cg)

2024-04-22 05:55:30.520275: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.0/lib64:/usr/local/cuda-11.0/lib64
2024-04-22 05:55:30.520409: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.0/lib64:/usr/local/cuda-11.0/lib64
2024-04-22 05:55:30.520507: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.0/lib64:/usr/local/cuda-11.0/lib64
2024-04-22 05:55:30.520602: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Cou

In [3]:
# Get pretrained weights
weights = model_pretrained.get_weights()
# The first weight is the element embedding
emb = weights[0] 
# The new model has exact the same layers except the embedding layer
model_new.set_weights(weights[1:]) 

In [4]:
from mp_api.client import MPRester

with MPRester() as mpr:
    s = mpr.get_structure_by_material_id('mp-129')

Retrieving MaterialsDoc documents: 100%|██████████| 1/1 [00:00<00:00, 5745.62it/s]


In [5]:
# Pretrained model prediction
model_pretrained.predict_structure(s)

array([0.03076017], dtype=float32)

In [6]:
# Concert manually the atomic number into an atomic feature
graph = model_new.graph_converter.convert(s)
graph['atom'] = [emb[i] for i in graph['atom']]

In [7]:
# Same prediction results
model_new.predict_graph(graph)

array([0.03076017], dtype=float32)